In [4]:
import ftplib
from ftplib import FTP
import os
from pathlib import Path
from datetime import datetime, timedelta

In [5]:
def create_missing_directories():
    # Define the path to the parent directory
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # Check if 'data' folder exists in the parent directory
    data_dir = os.path.join(parent_dir, 'data')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("'data' folder created in the parent directory.")
    else:
        print("'data' folder already exists in the parent directory.")

    # Check if 'satellite' directory exists inside 'data' folder
    satellite_dir = os.path.join(data_dir, 'satellite')
    if not os.path.exists(satellite_dir):
        os.makedirs(satellite_dir)
        print("'satellite' directory created inside 'data' folder.")
    else:
        print("'satellite' directory already exists inside 'data' folder.")

    # Check if 'NEODASS' directory exists inside 'satellite' folder
    NEODASS_dir = os.path.join(satellite_dir, 'NEODASS')
    if not os.path.exists(NEODASS_dir):
        os.makedirs(NEODASS_dir)
        print("'NEODASS' directory created inside 'satellite' folder.")
    else:
        print("'NEODASS' directory already exists inside 'satellite' folder.")

if __name__ == "__main__":
    create_missing_directories()


'data' folder already exists in the parent directory.
'satellite' directory already exists inside 'data' folder.
'NEODASS' directory already exists inside 'satellite' folder.


In [6]:
# Re-establish paths (should not be needed, but is...?)
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
NEODASS_dir = os.path.join(satellite_dir, 'NEODASS')

# Fixed FTP protocols (changes on a mission by mission basis)
hostname = "ftp.rsg.pml.ac.uk"
username = "neodaas17"
password = "Aeghingiech7fahd2vee3"

# Fixed start date (based on data availabiity) and end date set as current date, -1 day.
start_date = '2024-05-20'  # Start date in YYYY-MM-DD format
now = datetime.now()
formatted_date = now - timedelta(days = 1) 
end_date = formatted_date.strftime("%Y-%m-%d") # End date in YYYY-MM-DD format

start_date_loop = datetime.strptime(start_date, '%Y-%m-%d')
end_date_loop   = datetime.strptime(end_date, '%Y-%m-%d')

time_diff = end_date_loop - start_date_loop

##### Accessing files #####

for i in range(time_diff.days + 1):

    # Set directory
    current_date = start_date_loop + timedelta(days=i)
    ftp_dir      = f"large_area/{current_date.strftime('%Y/%m/%d')}"

    # Login to FTP server
    ftp_server = ftplib.FTP(hostname, username, password)

    try:
        # Move to directory
        ftp_server.cwd(ftp_dir)

        # List all files, trim to only netCDF files
        curdir_files = ftp_server.nlst()
        nc_files = [file for file in curdir_files if file.endswith('.nc')]

        # Download netCDF files
        for file in nc_files:
            local_filepath = os.path.join(NEODASS_dir, file)
            if os.path.exists(local_filepath):
                print(f"{local_filepath} already exists. Skipping download.")
                continue
            with open(local_filepath, 'wb') as local_file:
                ftp_server.retrbinary(f"RETR {file}", local_file.write)
            print(f"Downloaded {file} to {local_filepath}")

    except Exception as e:
        print(f"Could not change to directory {ftp_dir}: {e}")
        ftp_server.quit()
        continue

# Close connection
ftp_server.quit()

######

c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\NEODASS\cmems_sea_level-sla-1d-20240520_20240520-nrt.nc already exists. Skipping download.
c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\NEODASS\olci_a_1km-CHL_OC4ME-1d-20240520_20240520-nrt.nc already exists. Skipping download.
c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\NEODASS\olci_a_1km-CHL_OC4ME-7d-20240514_20240520-nrt.nc already exists. Skipping download.
c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\NEODASS\olci_b_1km-CHL_OC4ME-1d-20240520_20240520-nrt.nc already exists. Skipping download.
c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\NEODASS\slstr_a-sea_surface_temperature-1d-20240520_20240520-nrt.nc already exists. Skipping download.
c:\Users\hanshil\Documents\GitHub\biocarbon_nrt_data_viz\data\satellite\NEODASS\slstr_a-sea_surface_temperature-7d-20240514_20240520-nrt.nc already exists. Skipping downl

'221 Goodbye.'